In [6]:
import numpy as np
from pyswarm import pso

# 生成一个示例信号和噪声
def generate_signal(start_time, end_time, noise_level):
    t = np.linspace(0, 1, 1000)
    signal = np.zeros_like(t)
    signal[start_time:end_time] = np.sin(2 * np.pi * 5 * t[start_time:end_time])
    noise = noise_level * np.random.randn(len(t))
    return t, signal + noise

# 定义适应度函数
def fitness_function(x, t, signal_noise):
    start_time = int(x[0])
    end_time = int(x[1])
    
    # 确保结束时间大于开始时间
    if end_time <= start_time:
        return float('inf')  # 返回一个非常大的值，表示这个解不可行
    
    # 提取假设的信号段
    signal_segment = signal_noise[start_time:end_time]
    
    # 计算信号与噪声的比率（SNR）
    signal_power = np.mean(signal_segment**2)
    
    # 只考虑信号段之外的部分作为噪声
    noise_segment = np.concatenate((signal_noise[:start_time], signal_noise[end_time:]))
    noise_power = np.mean(noise_segment**2)
    
    snr = signal_power / noise_power if noise_power != 0 else float('inf')
    
    # 增加惩罚项，避免不合理的解
    penalty = 0
    if end_time - start_time < 50:  # 信号段长度小于50时增加惩罚
        penalty = 1000
    
    # 最大化SNR
    return -snr + penalty

# 生成信号和噪声数据
true_start_time = 200
true_end_time = 300
noise_level = 0.5
t, signal_noise = generate_signal(true_start_time, true_end_time, noise_level)

# 定义PSO的边界
lower_bound = [150, 250]  # 缩小搜索空间
upper_bound = [250, 350]  # 缩小搜索空间

# 使用PSO优化
best_positions = []
best_fitnesses = []
circulate_num = 8
for _ in range(circulate_num):
    best_position, best_fitness = pso(fitness_function, lower_bound, upper_bound, args=(t, signal_noise), swarmsize=200, maxiter=2000)
    best_positions.append(best_position)
    best_fitnesses.append(best_fitness)

# 选择最佳结果
best_index = np.argmin(best_fitnesses)
best_position = best_positions[best_index]

# 输出结果
estimated_start_time = int(best_position[0])
estimated_end_time = int(best_position[1])
print(f"Estimated start time: {estimated_start_time}")
print(f"Estimated end time: {estimated_end_time}")
print(f"True start time: {true_start_time}")
print(f"True end time: {true_end_time}")

Stopping search: maximum iterations reached --> 2000
Stopping search: maximum iterations reached --> 2000
Stopping search: maximum iterations reached --> 2000
Stopping search: maximum iterations reached --> 2000
Stopping search: maximum iterations reached --> 2000
Stopping search: maximum iterations reached --> 2000
Stopping search: maximum iterations reached --> 2000
Stopping search: maximum iterations reached --> 2000
Estimated start time: 220
Estimated end time: 270
True start time: 200
True end time: 300
